# Matlab to Python

This notebook walks through replicating the Matlab code in "Main_Yog_priceonly.m" and the associated functions. I have added notes before snippets of code where there are small differences with how code might appear in Matlab. 

There is also an associated script "matlab_to_python.py" ready to be run from a terminal. Besides putting the code in a more logical order, there are some other minor differences in that script.

### Import Modules

In [ ]:
from __future__ import division  # Necessary only in Python 2
import os

import numpy as np
import scipy.optimize as opt

In [ ]:
home = os.environ['HOME']
root = home + '/Dropbox/Instruction/Scraping/maria_ana/for_class'

### Set up Data

For data saved in standard formats, e.g. csv, where each row represents an observation, it is more efficient to use the module pandas. The resulting data structure in your program is something analagous to a Stata dataset. 

A numpy import, as demonstrated below, with more freely formatted data is closer to a Matlab import. The resulting data structure is like a vector or matrix in Matlab.

In [ ]:
yogurt = np.loadtxt(root + '/yogurt.txt')

**NB**: Indexing starts with "0" not "1". Additionally, the index is inclusive at the beginning but not inclusive at the end. 

In [ ]:
pan = yogurt[:, 0]  # Id Number of Panelists
price = yogurt[:, 14:18]  # Prices for the 4 brands
choi = yogurt[:, 6:10]  # Brand purchase information
n = yogurt.shape[0]  # Shape of the matrix along the first dimension; Equivalent to len(yogurt)
o = np.ones((n, 1))  # Create n by 1 matrix of ones; the first argument of the function is the shape

Two methods to concatenate Numpy objects are demonstrated below. In line 1, "price[:, 0]" generates a vector. In line 2, "price[:, [1]]" generates a matrix of size n by 1.

In [ ]:
yop = np.c_[o, price[:, 0]]  # Concatenate columns
dan = np.concatenate((o, price[:, [1]]), axis=1)  # Concatenation along the specified axis (here add a column)
hil = np.c_[o, price[:, 2]]
wwt = price[:, [3]]

### Estimate the Logit Model

First I define the log likelihood function to be minimized by the optimizer. Note the "\*" function is like ".\*" in Matlab. To do matrix multiplication on, say, A * B, the code is "A.dot(B)" (dot for dot product in 2d).

**NB**: In Python, variables assigned in the main script do not need to be referenced in functions defined within the program. The variables must be assigned before the function is called (not defined; for example see where functions are define in "matlab_to_python.py". These functions, however, cannot manipulate the variables without explicitly marking them as globals within the function.

In [ ]:
def loglike(param):
    e_y = np.exp(yop.dot(param[[0, 3]]))
    e_d = np.exp(dan.dot(param[[1, 3]]))
    e_h = np.exp(hil.dot(param[[2, 3]]))
    e_w = np.exp(wwt.dot(param[[3]]))
    
    den = e_y + e_d + e_h + e_w
    py = e_y / den
    pd = e_d / den
    ph = e_h / den
    pw = e_w / den
    p = np.c_[py, pd, ph, pw]
    selbmat = choi * p
    selb = selbmat.sum(axis=1)  # Sum collapsing the second dimension (columns)
    
    lselb = np.log(selb)
    lpr = -lselb.sum()
    
    return lpr

In [ ]:
X = -np.ones((4, 1))  # Initial values for parameters

The "minimize" function is the basic optimization tool in Python (from the scipy library). There are a number of methods that can be used with the method specified at the time of calling the functions.

In [ ]:
bfgs_options = {'maxiter': 100000, 
                'maxfun': 100000, 
                'ftol': 1e-5,
                'disp': False}

In [ ]:
results = opt.minimize(loglike, X, method='L-BFGS-B', options=bfgs_options)
xfinal = results['x']  # The results object is a dictionary of various outputs

In [ ]:
print(xfinal)

### Calculate the standard errors of the estimates

**NB**: Unlike in Matlab, in numpy assigning an array to another does not create a copy of that array. For example, if in line 7 below I assigned "param" to "bj" without copy, any changes to "bj", as in line 9, would also be applied to the original "param".

In [ ]:
def likfunction(param):
    e_y = np.exp(yop.dot(param[[0, 3]]))
    e_d = np.exp(dan.dot(param[[1, 3]]))
    e_h = np.exp(hil.dot(param[[2, 3]]))
    e_w = np.exp(wwt.dot(param[[3]]))
    
    den = e_y + e_d + e_h + e_w
    py = e_y / den
    pd = e_d / den
    ph = e_h / den
    pw = e_w / den
    p = np.c_[py, pd, ph, pw]
    selbmat = choi * p
    selb = selbmat.sum(axis=1)
    
    lselb = np.log(selb)
    
    return lselb

In [ ]:
def serrors_basic_logit(param):
    no_params = param.shape[0]
    H = np.zeros((no_params, no_params))
    di = np.zeros((n, no_params))
    
    for l in xrange(no_params):
        bj = param.copy()
        bj2 = param.copy()
        bj[l] = .05 + param[l]
        di[:, l] = (likfunction(bj2) - likfunction(bj)) / .05
        
    for i in xrange(n):
        H += di[[i], :].T.dot(di[[i], :]) 
        
    serrors = np.sqrt(np.diag(np.linalg.inv(H)))
    
    return serrors

In [ ]:
serrors = serrors_basic_logit(xfinal)
tstats = xfinal / serrors
print(np.c_[xfinal, serrors, tstats])